In [1]:
%load_ext autoreload
%autoreload 2
lang = "eng"

import sys
sys.path.insert(0, "../src/")
import numpy as np
import pandas as pd
import random
random.seed(2)
from copy import deepcopy
from collections import Counter
from scipy.stats import pearsonr
from math import sqrt
import heapq
import statistics
get_ipython().run_line_magic("matplotlib", "inline") # %matplotlib inline
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.linear_model import Lasso
from sklearn.svm import SVR, SVC
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import xgboost
from xgboost import XGBRegressor
from utils import read_sentiment_scores, read_library_scores
from cv import Regression, TwoclassClassification, LibraryClassification, MulticlassClassification

features_dir = f"../data/features-3/{lang}/" ##############################
results_dir = f"../data/results_sentiment/classification_library/{lang}/" ##############3############33
sentiment_labels_dir = "../data/labels_sentiment/"
canonization_labels_dir = "../data/labels_canon/"

labels = read_sentiment_scores(sentiment_labels_dir, canonization_labels_dir, lang)
library_scores = read_library_scores(sentiment_labels_dir, canonization_labels_dir, lang)

IndentationError: unindent does not match any outer indentation level (cv.py, line 649)

In [ ]:
labels

In [ ]:
## Labels statistics
print(len(pd.unique(labels["book_name"]))) #197
# 254 labels, 197 different book_names -> 57 second/third... reviews
# 36 book_names with more than 1 label, these 36 book_names have 93 labels
# 93 = 36 first reviews + 57 second/third... reviews
# 6 texts have opposing reviews (13 reviews are opposing)
# 191 texts after aggregating (without opposing reviews)

labels["y"].plot.hist(grid=True, bins=50)

In [ ]:
'''
Fehler
Brooke_Frances_Emily_Montague_1769
Lennox_Charlotte_The-Female_Quixote_1752
Stoker_Bram-Dracula_1897

Nicolai_Friedrich_Sebaldus-Nothanker-1773
Jung-Stilling_Heinrich-Stillings-Jugend_1777
Sacher-Masoch_Venus-im-Pelz_1869
Hunold_Christian_Friedrich_Die-liebenswuerdige-Adalie_1681

Hoffmansthal_Hugo ['Hoffmansthal_Hugo_Andreas-oder-die-Vereinigten_1907', 'Hoffmansthal_Hugo_Das-Maerchen-der-672-Nacht_1895'] 
Hoffmansthal_Hugo-von ['Hoffmansthal_Hugo-von_Ein-Brief_1902', 'Hoffmansthal_Hugo-von_Reitergeschichte_1899'] 


Anonymous anonymous

df = df.loc[df["book_name"] != "Defoe_Daniel_Roxana_1724"]

'''

In [ ]:
'''
Parameter combinations
'''
langs = ["eng", "ger"]
langs = ["eng"] ################################################3
models = ["svr", "lasso", "xgboost", "svc"]
model_params = {"svr": [1], "lasso": [1, 4], "xgboost": [None], "svc": [0.1, 1, 10, 100, 1000, 10000]} 
dimensionality_reduction = ["ss_pca_0_95", 'k_best_f_reg_0_10', 'k_best_mutual_info_0_10', [None]]
features = ["book", "chunk", "book_and_averaged_chunk", "chunk_and_copied_book"]

regression_dict = {"model": ["xgboost"], 
                     "dimensionality_reduction": dimensionality_reduction[-1], 
                     "features": [features[0]]}
library_dict = {"model": ["svc", "xgboost"], 
                  "dimensionality_reduction": dimensionality_reduction[-1], 
                  "features": ["book", "book_and_averaged_chunk"]}
twoclass_dict = {"model": ["svc", "xgboost"], 
                   "dimensionality_reduction": dimensionality_reduction[-1], 
                   "features": ["book", "book_and_averaged_chunk"]}
multiclass_dict = {"model": ["svc", "xgboost"], 
                     "dimensionality_reduction": dimensionality_reduction[-1],
                     "features": ["book", "book_and_averaged_chunk"]}
testing_dict = {"model": models[3], 
                  "dimensionality_reduction": dimensionality_reduction[-1], 
                  "features": ["book"]}

In [ ]:
'''
Run Cross-Validation
'''
results = []
target = "regression" 
if target == "regression":
    param_dict = regression_dict
elif target = "library":
    param_dict = library_dict
elif target == "twoclass":
    param_dict = twoclass_dict
elif target == "multiclass":
    param_dict = multiclass_dict
elif target == "testing":
    param_dict = testing_dict

book_df = pd.read_csv(f"{features_dir}book_df.csv")
book_and_averaged_chunk_df = pd.read_csv(f"{features_dir}book_and_averaged_chunk_df.csv")
chunk_df = pd.read_csv(f"{features_dir}chunk_df.csv")
chunk_and_copied_book_df = pd.read_csv(f"{features_dir}chunk_and_copied_book_df.csv")

#Remove duplicate text
book_df = book_df.loc[book_df["book_name"] != "Defoe_Daniel_Roxana_1724"] ########################################
book_and_averaged_chunk_df = book_and_averaged_chunk_df.loc[book_and_averaged_chunk_df["book_name"] != "Defoe_Daniel_Roxana_1724"]
chunk_df = chunk_df.loc[chunk_df["book_name"] != "Defoe_Daniel_Roxana_1724"]
chunk_and_copied_book_df = chunk_and_copied_book_df.loc[chunk_and_copied_book_df["book_name"] != "Defoe_Daniel_Roxana_1724"]


for lang in langs:
    drop_columns_list = [
        ["average_sentence_embedding", "100_most_common_", "doc2vec_chunk_embedding"],
        ["average_sentence_embedding", "100_most_common_", "doc2vec_chunk_embedding", "pos"],
        ]
    if lang == "eng":
        drop_columns_list.extend([
            ["average_sentence_embedding", "100_most_common_", "doc2vec_chunk_embedding", "->"], 
            ["average_sentence_embedding", "100_most_common_", "doc2vec_chunk_embedding", "->", "pos"]
        ])
        
    for model in [] + param_dict['model']:
        model_param = model_params[model]
        for model_param in model_param:
            for dimensionality_reduction in param_dict["dimensionality_reduction"]:
                for features in param_dict["features"]:
                    for drop_columns in drop_columns_list:
                        print(target, lang, model, features, drop_columns, dimensionality_reduction, 'param=', model_param)
                        if target == 'regression':
                            model_cols = [
                                "mean_train_mse", "mean_train_rmse", "mean_train_mae", "mean_train_r2", 
                                "mean_train_corr", "mean_validation_mse", "mean_validation_rmse", 
                                "mean_validation_mae", "mean_validation_r2", "mean_validation_corr", 
                                "mean_p_value"]
                            
                            experiment = Regression(
                                language=lang,
                                features=features,
                                drop_columns=drop_columns,
                                dimensionality_reduction = dimensionality_reduction,
                                model_param=model_param,
                                model=model,
                                verbose=True,
                                target = target)
                            
                        if target =="library":
                            labels = library_scores####################################3
                            model_cols = ["mean_train_book_acc", "mean_validation_book_acc"]
                            experiment = LibraryClassification(
                                language=lang,
                                features=features,
                                drop_columns=drop_columns,
                                dimensionality_reduction=dimensionality_reduction,
                                model_param=model_param,
                                model=model,
                                verbose=True,
                                target = target)
                            
                        elif target == 'twoclass':
                            model_cols = ["mean_train_book_acc", "mean_validation_book_acc"]
                            experiment = TwoclassClassification(
                                language=lang,
                                features=features,
                                drop_columns=drop_columns,
                                dimensionality_reduction=dimensionality_reduction,
                                model_param=model_param,
                                model=model,
                                verbose=True,
                                target = target)
                            
                        elif target == 'multiclass':
                            model_cols = ["mean_train_f1", "mean_validation_f1"]
                            experiment = MulticlassClassification(
                                language=lang,
                                features=features,
                                drop_columns=drop_columns,
                                dimensionality_reduction=dimensionality_reduction,
                                model_param=model_param,
                                model=model,
                                verbose=True,
                                target = target)
                            
                        returned_results = experiment.run()
                        results.append([lang, model, features, drop_columns, dimensionality_reduction, 
                                        model_param] + returned_results) 
                                        
                        print(lang, model, features, drop_columns, dimensionality_reduction, model_param, 
                              returned_results)
                        print('\n-----------------------------------------------------------\n')


    results_df = pd.DataFrame(results, columns=['lang', 'model', "features", "drop_columns", 
                                                "dimensionality_reduction", "model_param"] + model_cols)
    results_df.to_csv(f"{results_dir}results-{lang}-{target}'.csv", index=False)